## 卷积神经网络

> 卷积，相比于全连接的优劣势

> 感受野是相对与原始输入来说的

### 卷积的演变
> 自动控制原理中的卷积：  
> **输出就是过去输入按某种权重 $h(\cdot)$ 在时间上滑动叠加的结果，这个叠加就是卷积。**

$$
y(t) = x(t) * h(t)
     = \int_{-\infty}^{+\infty} x(\tau)\,h(t-\tau)\,d\tau
$$

其中 $x(t)$ 是输入信号，$h(t)$ 是该线性时不变系统的单位冲激响应。

> 线性时不变系统中，单位冲激响应就是传递函数的反拉普拉斯变换（一阶惯性系统为例）：

$$
\mathcal{L}^{-1}\left\{\frac{1}{s + a}\right\} = e^{-at} u(t)
$$

$$
G(s) = \frac{1}{Ts + 1}
      = \frac{1}{T}\cdot\frac{1}{s + \frac{1}{T}}
\Rightarrow
h(t) = \frac{1}{T} e^{-t/T} u(t)
$$

> 以一阶惯性系统对单位阶跃输入的卷积为例：

$$
\begin{aligned}
y(t) &= x(t) * h(t)
     = \int_{-\infty}^{+\infty} u(\tau)\,\frac{1}{T} e^{-\frac{(t-\tau)}{T}} u(t-\tau)\,d\tau \\
     &= \int_{0}^{t} \frac{1}{T} e^{-\frac{(t-\tau)}{T}}\,d\tau
      = 1 - e^{-t/T}, \quad t \ge 0
\end{aligned}
$$

> 也可以把卷积理解成“时间窗 + 加权叠加”的过程。  
> 以一阶惯性系统接收一个宽度为 1 的方波输入为例：
> 令
$$
x(t) = w(t) =
\begin{cases}
1, & 0 \le t \le 1\\[2pt]
0, & \text{其他}
\end{cases}
$$
> 这个 $w(t)$ 本质上就是一个宽度为 1 的矩形窗（窗函数），表示“系统只在 $[0,1]$ 这一秒钟内接收到输入，其他时刻都为 0”。

> 对这个输入的卷积为：
 $$
y(t) = (w * h)(t)
= \int_{-\infty}^{+\infty} w(\tau)\,h(t-\tau)\,d\tau
$$
>由于 $w(\tau)$ 只在 $[0,1]$ 非零，积分会被“窗”截断：
> - 当 $0 \le t \le 1$ 时：
$$
 y(t) = \int_0^t \frac{1}{T} e^{-\frac{(t-\tau)}{T}}\,d\tau
      = 1 - e^{-t/T}
 $$
>   系统正在“积累”这 1 秒内持续输入的影响；
> - 当 $t > 1$ 时：
 $$
 y(t) = \int_0^1 \frac{1}{T} e^{-\frac{(t-\tau)}{T}}\,d\tau
      = (e^{1/T}-1)\,e^{-t/T}
 $$
>   输入已经关掉（窗外为 0），输出只剩下系统对这 1 秒输入的“余辉”，按 $h(t)$ 的形状缓慢衰减。

> 从这个角度看，窗函数 $w(t)$ 负责“挑出”某一段时间内的输入，再由卷积核 $h(\cdot)$ 对这段历史进行加权叠加，这和深度学习中卷积核在时间或空间上滑动，对一个局部窗内的像素（或特征）做加权求和，在本质上是同一个思想。



In [2]:
from mxnet import autograd, nd
from mxnet.gluon import nn
# 2d卷积的实现
def corr2d(X, K):
    h, w = K.shape
    Y = nd.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range( Y.shape[1]):
            Y[i,j] = (X[i:i+h, j:j+w]*K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0,1], [2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

### 边缘提取最基础的小例子

In [13]:
# 定义一个核
K = nd.array([[1, -1]])
# 假设一个图像
X = nd.ones((6, 6))
X[:, 2:4] = 0
# 边缘提取，相同的为0，不相同的为非0
# 有效的表征局部特征
Y = corr2d(X, K)

In [27]:
# 简单训练一个卷积层，实际就是在训练卷积核参数
conv2d = nn.Conv2D(1,kernel_size=(1, 2))
conv2d.initialize()
# reshape = 不改数据，只改“你怎么切片看它”的方式
X = X.reshape((1, 1, 6, 6))
Y = Y.reshape((1, 1, 6, 5))

epoch = 100
# for i in range(epoch):
#     with autograd.record():
#         y_hat = conv2d(X)
#         l = (y_hat - Y)**2
#     l.backward()
#     conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
#     if (i + 1) % 2 == 0:
#         print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))
for i in range(epoch):
    with autograd.record():
        y_hat = conv2d(X)
        l = ((y_hat - Y)**2).mean()
    l.backward()
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    conv2d.bias.data()[:] -= 3e-2 * conv2d.bias.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 0.401
batch 4, loss 0.383
batch 6, loss 0.365
batch 8, loss 0.347
batch 10, loss 0.331
batch 12, loss 0.315
batch 14, loss 0.300
batch 16, loss 0.286
batch 18, loss 0.273
batch 20, loss 0.260
batch 22, loss 0.248
batch 24, loss 0.236
batch 26, loss 0.225
batch 28, loss 0.214
batch 30, loss 0.204
batch 32, loss 0.195
batch 34, loss 0.185
batch 36, loss 0.177
batch 38, loss 0.168
batch 40, loss 0.160
batch 42, loss 0.153
batch 44, loss 0.146
batch 46, loss 0.139
batch 48, loss 0.132
batch 50, loss 0.126
batch 52, loss 0.120
batch 54, loss 0.114
batch 56, loss 0.109
batch 58, loss 0.104
batch 60, loss 0.099
batch 62, loss 0.094
batch 64, loss 0.090
batch 66, loss 0.086
batch 68, loss 0.082
batch 70, loss 0.078
batch 72, loss 0.074
batch 74, loss 0.071
batch 76, loss 0.067
batch 78, loss 0.064
batch 80, loss 0.061
batch 82, loss 0.058
batch 84, loss 0.055
batch 86, loss 0.053
batch 88, loss 0.050
batch 90, loss 0.048
batch 92, loss 0.046
batch 94, loss 0.044
batch 96, loss 0.

In [29]:
conv2d.weight.data().reshape((1, 2))
#conv2d.bias.data().reshape((1,))


[[ 0.697463  -0.6961364]]
<NDArray 1x2 @cpu(0)>

### 填充和步幅
> 当核为大小为偶数时，如何实现两侧填充不一样
padding（0，1）默认是两侧填充一样，不过是高两侧填充0，宽填充1
### 通道
> 我有 3 通道输入，想变成 2 通道输出，那么有 2 个 3*h*w 形状的核

In [ ]:
def corr2d_multi_in(X, K):
    # zip 的行为 和 对 NDArray 迭代时默认按第 0 维切片
    # 在in 后面的都是可迭代对象
    # []变成列表推导式
    # * 是把列表拆开当作多个参数传进去，相当于：
    # Ys = [d2l.corr2d(x, k) for x, k in zip(X, K)]
    # nd.add_n(Ys[0], Ys[1], Ys[2], ...)
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])

## 1×1 卷积（1×1 卷积 = 对每个像素做一次 C_in → C_out 的全连接）：
矩阵视角（3×H×W → 2×H×W）

设 $N = H \times W$。

### 1. 展平空间维度

输入：
$$
X \in \mathbb{R}^{3 \times H \times W}
$$

展平为：
$$
X' \in \mathbb{R}^{3 \times N}
$$

$$
X' =
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,N} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,N} \\
x_{3,1} & x_{3,2} & \cdots & x_{3,N}
\end{bmatrix}
$$

---

### 2. 1×1 卷积核（2 个输出通道）

$$
W \in \mathbb{R}^{2 \times 3},\quad
W =
\begin{bmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23}
\end{bmatrix}
$$

---

### 3. 矩阵乘法（对每个样本做 3→2 变换）

$$
Y' = W X' \in \mathbb{R}^{2 \times N}
$$

$$
Y' =
\begin{bmatrix}
y_{1,1} & y_{1,2} & \cdots & y_{1,N} \\
y_{2,1} & y_{2,2} & \cdots & y_{2,N}
\end{bmatrix}
$$

---

### 4. 恢复空间形状

$$
Y' \Rightarrow Y \in \mathbb{R}^{2 \times H \times W}
$$
